# 03 – Preparação do RAG STRIDE/DREAD com Docling

Este notebook prepara o conteúdo para o RAG dos agentes **STRIDE** e **DREAD**. Usamos a biblioteca [Docling](https://github.com/docling-project/docling) para processar fontes em diversos formatos (PDF, DOCX, Markdown, imagens, etc.) e gerar Markdown estruturado.

## Objetivo

Pegar **todos os arquivos** que tivermos sobre conhecimento de STRIDE e DREAD (PDF, Markdown, DOCX, sites exportados, imagens de infográficos, vídeos transcritos, etc.) e usar o Docling para:

1. Converter formatos complexos (PDF, DOCX) em Markdown limpo
2. Separar o conteúdo por tema: **STRIDE** e **DREAD**
3. Gerar arquivos em `output_files/stride/` e `output_files/dread/` para uso no RAG

Assim, os agentes especializados (StrideAgent e DreadAgent) terão uma base de conhecimento enriquecida, gerando análises mais precisas.

## 1. Instalação do Docling

Se ainda não tiver o Docling instalado, execute na célula abaixo ou no terminal:

```bash
pip install docling
```

In [1]:
from docling.document_converter import DocumentConverter

## 2. Estrutura de Pastas

A base de conhecimento está organizada em:

- **input_files/stride/** — Fontes sobre STRIDE (PDF, MD, DOCX, imagens)
- **input_files/dread/** — Fontes sobre DREAD
- **output_files/stride/** — Markdown processado para o StrideAgent
- **output_files/dread/** — Markdown processado para o DreadAgent

In [2]:
from pathlib import Path
import shutil

PROJECT_ROOT = Path.cwd()
while PROJECT_ROOT != PROJECT_ROOT.parent and not (PROJECT_ROOT / "notebooks").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent

KB_DIR = PROJECT_ROOT / "docs" / "knowledge-base"
INPUT_STRIDE = KB_DIR / "input_files" / "stride"
INPUT_DREAD = KB_DIR / "input_files" / "dread"
OUTPUT_STRIDE = KB_DIR / "output_files" / "stride"
OUTPUT_DREAD = KB_DIR / "output_files" / "dread"

OUTPUT_STRIDE.mkdir(parents=True, exist_ok=True)
OUTPUT_DREAD.mkdir(parents=True, exist_ok=True)

print(f"Input STRIDE: {INPUT_STRIDE}")
print(f"Input DREAD:  {INPUT_DREAD}")
print(f"Output STRIDE: {OUTPUT_STRIDE}")
print(f"Output DREAD:  {OUTPUT_DREAD}")

Input STRIDE: /home/lucas-biason/Projetos/Projetos/Ativos/threat-modeling-ai/docs/knowledge-base/input_files/stride
Input DREAD:  /home/lucas-biason/Projetos/Projetos/Ativos/threat-modeling-ai/docs/knowledge-base/input_files/dread
Output STRIDE: /home/lucas-biason/Projetos/Projetos/Ativos/threat-modeling-ai/docs/knowledge-base/output_files/stride
Output DREAD:  /home/lucas-biason/Projetos/Projetos/Ativos/threat-modeling-ai/docs/knowledge-base/output_files/dread


## 3. Formatos Suportados pelo Docling

O Docling processa: PDF, DOCX, PPTX, XLSX, HTML, imagens (PNG, JPEG, TIFF), etc. Para arquivos **Markdown** (`.md`), copiamos diretamente para a saída, pois já estão no formato final.

In [3]:
DOCLING_EXTENSIONS = {'.pdf', '.docx', '.pptx', '.xlsx', '.html', '.htm', '.png', '.jpg', '.jpeg', '.tiff'}
MARKDOWN_EXTENSIONS = {'.md'}

def needs_docling(path: Path) -> bool:
    return path.suffix.lower() in DOCLING_EXTENSIONS

def is_markdown(path: Path) -> bool:
    return path.suffix.lower() in MARKDOWN_EXTENSIONS

## 4. Processamento com Docling

Para cada arquivo de entrada:

- **PDF, DOCX, etc.:** Docling converte e exporta para Markdown
- **Markdown:** Copia para a pasta de saída

Referência: [Docling GitHub](https://github.com/docling-project/docling), [ottomator docling-rag-agent](https://github.com/coleam00/ottomator-agents/tree/main/docling-rag-agent)

In [4]:
from docling.document_converter import DocumentConverter

converter = DocumentConverter()

def process_file(source: Path, output_dir: Path) -> bool:
    """Processa um arquivo: Docling para formatos complexos, cópia para MD."""
    try:
        if is_markdown(source):
            dest = output_dir / source.name
            shutil.copy2(source, dest)
            print(f"  OK Copiado MD: {source.name}")
            return True
        if needs_docling(source):
            result = converter.convert(str(source))
            md_content = result.document.export_to_markdown()
            dest = output_dir / (source.stem + ".md")
            dest.write_text(md_content, encoding="utf-8")
            print(f"  OK Convertido: {source.name} -> {dest.name}")
            return True
    except Exception as e:
        print(f"  ERRO em {source.name}: {e}")
        return False
    return False

print("Função process_file definida.")

Função process_file definida.


## 5. Processar STRIDE e DREAD

Percorre `input_files/stride/` e `input_files/dread/`, processa cada arquivo e grava em `output_files/stride/` e `output_files/dread/`.

In [ ]:
def process_folder(input_dir: Path, output_dir: Path, label: str):
    if not input_dir.exists():
        print(f"{label}: pasta {input_dir} não existe.")
        return
    files = [f for f in input_dir.iterdir() if f.is_file() and not f.name.startswith(".")]
    print(f"\n--- {label} ({len(files)} arquivos) ---")
    for f in sorted(files):
        if needs_docling(f) or is_markdown(f):
            process_file(f, output_dir)
        else:
            print(f"  Ignorado (formato nao suportado): {f.name}")

process_folder(INPUT_STRIDE, OUTPUT_STRIDE, "STRIDE")
process_folder(INPUT_DREAD, OUTPUT_DREAD, "DREAD")

print("\nProcessamento concluido. Saidas em output_files/stride e output_files/dread.")


--- STRIDE (13 arquivos) ---


[INFO] 2026-02-05 15:54:02,858 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2026-02-05 15:54:02,884 [RapidOCR] download_file.py:60: File exists and is valid: /home/lucas-biason/Projetos/Projetos/Ativos/threat-modeling-ai/.venv/lib/python3.13/site-packages/rapidocr/models/ch_PP-OCRv4_det_infer.onnx
[INFO] 2026-02-05 15:54:02,888 [RapidOCR] main.py:53: Using /home/lucas-biason/Projetos/Projetos/Ativos/threat-modeling-ai/.venv/lib/python3.13/site-packages/rapidocr/models/ch_PP-OCRv4_det_infer.onnx
[INFO] 2026-02-05 15:54:03,530 [RapidOCR] base.py:22: Using engine_name: onnxruntime
[INFO] 2026-02-05 15:54:03,543 [RapidOCR] download_file.py:60: File exists and is valid: /home/lucas-biason/Projetos/Projetos/Ativos/threat-modeling-ai/.venv/lib/python3.13/site-packages/rapidocr/models/ch_ppocr_mobile_v2.0_cls_infer.onnx
[INFO] 2026-02-05 15:54:03,546 [RapidOCR] main.py:53: Using /home/lucas-biason/Projetos/Projetos/Ativos/threat-modeling-ai/.venv/lib/python3.13/site-packages/ra

## 6. Próximos Passos

1. **Validar** os Markdown gerados em `output_files/`
2. **Configurar o backend** para usar `output_files/` (ou unir stride + dread) como `KNOWLEDGE_BASE_PATH` no RAG
3. **Implementar RAG** nos agentes StrideAgent e DreadAgent (após validação dos notebooks)

O backend carrega os `.md` da base de conhecimento para criar embeddings (ChromaDB) e recuperar chunks relevantes para cada query do LLM.